In [3]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.io import MemoryFile

from vayu_gnn.dbx.dbx_config import dbx_helper, DropboxHelper

In [1]:
def combine_and_upload_tifs(tif1, tif2, directory: str, filename: str):
    """
    Combines two in-memory TIFF files (side-by-side) and uploads the resulting mosaic to Dropbox.
    
    Args:
        tif1 (BytesIO): The first TIFF file in-memory.
        tif2 (BytesIO): The second TIFF file in-memory.
        directory (str): The directory within the base Dropbox path where the file will be saved.
        filename (str): The name of the output combined TIFF file.
    """
    if tif1 is None or tif2 is None:
        print("Failed to load one of the files.")
        return

    # Open the in-memory files using MemoryFile and merge them
    with MemoryFile(tif1) as memfile1, MemoryFile(tif2) as memfile2:
        with memfile1.open() as src1, memfile2.open() as src2:
            # Merge the two datasets into one mosaic
            mosaic, out_trans = merge([src1, src2])
            
            # Update the metadata based on the merged raster
            out_meta = src1.meta.copy()
            out_meta.update({
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "driver": "GTiff",
                "compress": "lzw"  # Using LZW compression to reduce file size
            })

    # Write the mosaic to an in-memory file (BytesIO)
    with MemoryFile() as memfile:
        with memfile.open(**out_meta) as dest:
            dest.write(mosaic)
        memfile.seek(0)  # Ensure we are at the start of the file
        combined_tif_bytes = memfile.read()

    # Upload the combined TIFF to Dropbox using your existing helper function
    dbx_helper.write_bytes(combined_tif_bytes, dbx_helper.raw_input_path, directory, filename)

In [4]:
# Total (non-residential + residential) building area
tif_1 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/total', 'GHS_BUILT_S_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C26.tif')
tif_2 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/total', 'GHS_BUILT_S_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C27.tif')

combine_and_upload_tifs(tif_1, tif_2, 'settlement', 'GHS_total_building_area.tif')

# Total (non-residential + residential) building volume
tif_1 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/total', 'GHS_BUILT_V_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C26.tif')
tif_2 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/total', 'GHS_BUILT_V_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C27.tif')

combine_and_upload_tifs(tif_1, tif_2, 'settlement', 'GHS_total_building_volume.tif')

# Non-residential building area
tif_1 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/non_residential', 'GHS_BUILT_S_NRES_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C26.tif')
tif_2 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/non_residential', 'GHS_BUILT_S_NRES_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C27.tif')

combine_and_upload_tifs(tif_1, tif_2, 'settlement', 'GHS_non_res_building_area.tif')

# Non-residential building volume
tif_1 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/non_residential', 'GHS_BUILT_V_NRES_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C26.tif')
tif_2 = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement/non_residential', 'GHS_BUILT_V_NRES_E2025_GLOBE_R2023A_54009_100_V1_0_R6_C27.tif')

combine_and_upload_tifs(tif_1, tif_2, 'settlement', 'GHS_non_res_building_volume.tif')


File 'GHS_total_building_area.tif' successfully uploaded to Dropbox path: '/input/raw/settlement/GHS_total_building_area.tif'
File 'GHS_total_building_volume.tif' successfully uploaded to Dropbox path: '/input/raw/settlement/GHS_total_building_volume.tif'
File 'GHS_non_res_building_area.tif' successfully uploaded to Dropbox path: '/input/raw/settlement/GHS_non_res_building_area.tif'
File 'GHS_non_res_building_volume.tif' successfully uploaded to Dropbox path: '/input/raw/settlement/GHS_non_res_building_volume.tif'
